## CRISM Image Preprocessing [ and dependencies]

1. Download lab spectra AND preprocess using USGS_preprocessing_helpers 
- for Mars/CRISM testing: Have lab_spectra/USGS (as currently available)
- for Moon/M3 testing: Have lab_spectra/RELAB

In [ ]:
from USGS_preprocessing_helpers import preprocess_USGS
preprocess_USGS()

2. Download planetary data. For CRISM, download from Mars Orbital Data Explorer, Filter on "Targeted TRDR (FRT, HRL, HRS FRS, ATO, ATU)  (center swath only I/F )". Filter to desired region using Map Search, then download.
- Do all preprocessing below for Mars data

	1. Open as CRISM files in CAT ENVI. 
		- ENVI Classic > Cat 7.4 > Open CRISM File > 
			i. …_07_if165l_trr3.img
			ii. …_07_if165s_trr3.img
	2. Rotate 180 degrees. Save as ENVI standard. This will match its appearance on Mars Orbital Data Explorer (so I can relate it to the original map)
		- ENVI Classic > Basic Tools > Rotate/Flip Data
		- Save as l_rotated.img or s_rotated.img
        
	3. Save pixel wavelengths in rotated file. (s_pixel, l_pixel) (Note: HAVE TO save pixel wavelengths after rotation, because rotation changes bands. Originally in one image there are 438 and after rotation there are 434.
		- Tools > Profiles > Z Profile (Spectrum ) 
		- File > Save Plot As > ASCII 
        - Open ASCII in Excel to clean it up, so we only save wavelengths as one column. Save as lpixel.csv or spixel.csv 


3. Specify CRISM image

In [ ]:
############## SPECIFY CRISM IMAGE HERE ##############
from utils.constants import *

CUR_IMG = "frt00010628_07"

CRISM_DIR = PREPROCESSED_DATA + "CRISM/" + CUR_IMG + "/"
# CRISM_DIR = PREPROCESSED_DATA + "CRISM/frs0002edd6_01/"

#######################################################

4. Layer s and l images into single CRISM image file (and save as Pickle). And replace all NULL values with 0.

In [ ]:
import pandas as pd
import numpy as np
from preprocessing.CRISM_preprocess_helpers import *
from utils.plotting import *

record_layered_data(img_dir=CRISM_DIR, img_save_name=CUR_IMG+"_LAYERED")


5. Reduced wavelengths: RW_CRISM and RW_USGS.
CRISM data has a certain wavelengths, and USGS has different ones. So, we need to find the mutually compatible set. This is what reducing wavelengths is.

When running the model, it will use these reduced wavelengths for the CRISM data and USGS endmembers. Saved in PREPROCESSED_DATA/CRISM


In [ ]:
CRISM_reduced, USGS_reduced = record_CRISM_USGS_reduced_wavelengths(CRISM_img_dir = CUR_IMG)


7. Calculate optical constants, k, for each endmember (in each wavelength) 

Usually we run this on the server/HPC because it takes a while (and can use many threads to infer k in different wavelengths at the same time).

In [ ]:
from preprocessing.estimatek import estimate_all_USGS_k
# True means to match CRISM wavelengths
estimate_all_USGS_k(True)

8. [OPTIONAL] Save PDF of image

In [ ]:
with open(CRISM_DIR + CUR_IMG + "_LAYERED.pickle", 'rb') as handle:
    img = pickle.load(handle)
plot_CRISM_img(img, CRISM_DIR + "frt00010628.pdf")


9. [OPTIONAL] Identify subsection of image to use for testing
Save as Pickle and as PDF for viewing.

In [ ]:

# rows
rowstart = 40
rowend = 60

#  cols
colstart = 80
colend = 100

coords = [ colstart, colend, rowstart, rowend]

# For plotting below, X corresponds to columns

plot_zoomed_sectioned_CRISM(img, coords, CRISM_DIR+"subsection.pdf")

SUBIMG = img[rowstart:rowend,colstart:colend, :]
view = imshow(SUBIMG,  bands= (100, 71, 18), title="Section I/F")

In [ ]:
# Save called-out section.
with open(CRISM_DIR + "subsection.pickle", 'wb') as F:
    pickle.dump(SUBIMG, F)